# Week 09 Group 40

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import random

##Generate our Dataset and Preprocess


In [ ]:
class gen_class(object):


    def __init__(self, n ):
        self.seq_length = n
        self.dataset_length = 0
        self.sequence_to_choose_from = range(1,10)
        self.seq = []


    def __iter__(self):
        return self


    def __next__(self):
        return self.next()

    def next(self):
        if self.dataset_length < 64*64: 
            self.dataset_length +=1
            q1 = random.choice(self.sequence_to_choose_from)
            q2 = random.choice(self.sequence_to_choose_from)
            # sample until the queried digits arent the same
            while (q1 == q2):
              q2 = random.choice(self.sequence_to_choose_from)
            self.seq = []
            for _ in range(self.seq_length):
              self.seq.append([q1,q2,random.choice(self.sequence_to_choose_from)])
            #append relevant queries
            
            target1 = self.seq.count([q1,q2,q1])
            target2 = self.seq.count([q1,q2,q2])
            if(target1 < target2):
                target = 1
            elif(target2 < target1):
                target = 2
            elif(target1 == target2):
                target = 0
            target = tf.one_hot(target,3)
            return  (tf.reshape(tf.convert_to_tensor(self.seq), [25,3]), tf.convert_to_tensor(target))
        raise StopIteration()

In [ ]:

ds = tf.data.Dataset.from_generator(generator=gen_class, output_shapes=([25,3],None) ,output_types=(tf.dtypes.float32, tf.dtypes.float32), args= [25])
ds = (ds.cache().shuffle(buffer_size = 10000, reshuffle_each_iteration=True).batch(64, )).prefetch(tf.data.experimental.AUTOTUNE)
print(ds)

<PrefetchDataset shapes: ((None, 25, 3), <unknown>), types: (tf.float32, tf.float32)>


##LSTM implementation

###LSTM_Cell

In [ ]:
class LSTM_Cell(tf.keras.Model):
    def __init__(self):
        super(LSTM_Cell, self).__init__()
        self.input_units = 3
        self.cell_state = tf.zeros([64,self.input_units],tf.dtypes.float32)
        self.forget_gate = tf.keras.layers.Dense(self.input_units, activation = "sigmoid", bias_initializer= "ones")
        self.input_gate = tf.keras.layers.Dense(self.input_units, activation = "sigmoid")
        self.cell_state_candidates = tf.keras.layers.Dense(self.input_units, activation = "tanh")
        self.output_gate = tf.keras.layers.Dense(self.input_units, activation = "sigmoid")

    #@tf.function    
    def __call__(self,input, hs, Training = True):
        # concatenate the input of our current timestep with our current hidden state
        concat_input = tf.concat([input, hs], axis = -1)

        # calculate the forget, input and output filter aswell as our new cell-state-candidates based on the concatenated input
        forget_filter = self.forget_gate(concat_input)
        input_filter = self.input_gate(concat_input)
        candidates = self.cell_state_candidates(concat_input)
        output_filter = self.output_gate(concat_input)

        #calculate new cellstate from our filters and old cellstate
        
        self.cell_state = forget_filter * self.cell_state + input_filter* candidates
        
        #calculate new hidden-state/output based on our new cellstate and output_filter
                #(1,1)              (1,4)                                 (4,1)
        hs = output_filter * tf.keras.activations.tanh(self.cell_state)
        return tf.convert_to_tensor(hs)


###LSTM

In [ ]:
class LSTM(tf.keras.Model):

  def __init__(self):

    super(LSTM, self).__init__()
    self.input_units = 3

    self.input_layer = tf.keras.layers.Dense(30, activation = "sigmoid")
    self.lstm_layer = LSTM_Cell()
    self.output_layer = tf.keras.layers.Dense(3, activation = "softmax")

  #@tf.function
  def call(self, input, hs, training = True):

    input = self.input_layer(input)
    for idx in tf.range(input.shape[1]):
      hs = self.lstm_layer(input[:,idx,:],hs)
    hs = self.output_layer(hs)
    return hs

##Training the Model

In [ ]:
#@tf.function
def train_step(model, input, target, loss_function, optimizer, training = True):
    with tf.GradientTape() as tape:
        hs = tf.zeros([input.shape[0],input.shape[2]])
        expectation = model(input, hs, training)        
        loss = loss_function(target, expectation)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_accuracy = tf.cast(tf.argmax(target, axis = 1) == tf.argmax(expectation, axis = 1), tf.int16)
    return loss, train_accuracy

In [ ]:
num_epochs = 25
learning_rate = 0.0005
running_average_factor = 0.95
# use categorical crossentropy as loss function
loss_function = tf.keras.losses.CategoricalCrossentropy()
# use adam as optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate)
# initialize the model
model = LSTM()

# lists for plotting later
train_loss_list = []
train_accuracy_list = []

# we train for the predefined number of epochs
for step in range(num_epochs):
    # reset average for each step
    changing_average_loss = 0
    # train the model with the training data
    for (input, target) in ds.take(100):
        # perform training step and store the loss and accuracy of step
        train_loss, train_accuracy = train_step(model, input, target, loss_function, optimizer)
        changing_average_loss = running_average_factor * changing_average_loss + (1 - running_average_factor) * train_loss
    train_loss_list.append(changing_average_loss.numpy())
    train_accuracy_list.append(np.mean(train_accuracy))
    
    # print values during training
    print("Epoch: " + str(step + 1))
    print("Training Accuracy: " + str(train_accuracy_list[step]))
    print("Train Loss: " + str(train_loss_list[step]))
    print()

Epoch: 1
Training Accuracy: 0.34375
Train Loss: 1.1295981

Epoch: 2
Training Accuracy: 0.4375
Train Loss: 1.009766

Epoch: 3
Training Accuracy: 0.390625
Train Loss: 0.9927715

Epoch: 4
Training Accuracy: 0.4375
Train Loss: 0.98774403

Epoch: 5
Training Accuracy: 0.4375
Train Loss: 0.99136287

Epoch: 6
Training Accuracy: 0.484375
Train Loss: 0.9964152

Epoch: 7
Training Accuracy: 0.484375
Train Loss: 0.993147

Epoch: 8
Training Accuracy: 0.40625
Train Loss: 0.99060357

Epoch: 9
Training Accuracy: 0.40625
Train Loss: 0.99102557

Epoch: 10
Training Accuracy: 0.578125
Train Loss: 0.9922195

Epoch: 11
Training Accuracy: 0.40625
Train Loss: 0.998401

Epoch: 12
Training Accuracy: 0.484375
Train Loss: 0.9935434

Epoch: 13
Training Accuracy: 0.484375
Train Loss: 0.9929112

Epoch: 14
Training Accuracy: 0.515625
Train Loss: 0.9975654

Epoch: 15
Training Accuracy: 0.5
Train Loss: 0.9939903

Epoch: 16
Training Accuracy: 0.4375
Train Loss: 0.99962914

Epoch: 17
Training Accuracy: 0.453125
Train Loss

##Training-task

###truncated BPTT

- No we should probably not use t-BPTT since our input-sequences always come in the same sizes, so we do not need to normalize them.



###Regression or Classification

- We should take this task as a classification task since we just want to assign three possible lables: 
1. The first shown context-number appears most often.
2. The second shwon context-number appears most often.
3. Both numbers appear the same amount of time.
Also getting typical regression predicitons do not make sense for this task and do not allow us to learn from them.